In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, QuatE
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive_io.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True
                     )


In [5]:
model_name = 'quatE_transductive_io'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200

In [8]:
model = QuatE(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Number of parameters: 13161600
Space occupied: 52646400 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory quatE_transductive_io created successfully!


In [10]:
learning_rate = 5e-2 #1e-3 stops after 20 iterations and actually does not reduce the error too much
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 4.530289173126221
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.001342324185970454
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0019431819126268534
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.001642753049298654
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0013423241907730699
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0019431819673627613
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0016427530208602545
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0013423241795189479
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.001943181911998677
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0016427530457588128
Metric: validation.head.optimistic.hits_at_1 = 0.00019462826002335538
Metric: validation.tail.optimistic.hits_at_1 = 0.00038925652004671076
Metric: validation.both.optimistic.hits_

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 4.534466743469238
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0007465175550328224
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0013883431704207402
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0010674303627267814
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0007465170347131789
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0013883431674912572
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0010674301302060485
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0007465165561831219
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0013883431456531562
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0010674298509181393
Metric: validation.head.optimistic.hits_at_1 = 9.731413001167769e-05
Metric: validation.tail.optimistic.hits_at_1 = 0.0004865706500583885
Metric: validation.both.optimistic.hits

...

Step: 109
Metric: loss = 0.08818119019269943
Step: 109
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.14989799851763758
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.22500119097408167
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.18744959474585965
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.14989794790744781
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.22500120103359225
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.18744957447052002
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.14989792899254775
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.22500119096478602
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.18744955997866689
Metric: validation.head.optimistic.hits_at_1 = 0.09565978980147917
Metric: validation.tail.optimistic.hits_at_1 = 0.1436356558972363
Metric: validation.both.optimistic.hits_at_1 = 0.1196477

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 110
Metric: loss = 0.08968324214220047
Step: 110
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.14826270227275443
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.22369336660662764
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.18597803443969102
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.14826270937919617
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.2236933708190918
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.18597804009914398
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.1482627018748431
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.22369336660662764
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.1859780342407354
Metric: validation.head.optimistic.hits_at_1 = 0.09322693655118723
Metric: validation.tail.optimistic.hits_at_1 = 0.14275982872713117
Metric: validation.both.optimistic.hits_at_1 = 0.117993382

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 111
Metric: loss = 0.08576033264398575
Step: 111
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.14802075896511815
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.22166763547467472
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.18484419721989645
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.14802077412605286
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.22166761755943298
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.18484419584274292
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.14802075894021297
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.22166763531325342
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.1848441971267332
Metric: validation.head.optimistic.hits_at_1 = 0.09303230829116388
Metric: validation.tail.optimistic.hits_at_1 = 0.14159205916699105
Metric: validation.both.optimistic.hits_at_1 = 0.1173121

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0    4.530289
1    4.534467
2    4.535563
3    4.535436
4    4.532721
..        ...
106  0.089325
107  0.091649
108  0.088181
109  0.089683
110  0.085760

[111 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/41.5k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.870403   0.870403     0.870403
hits_at_1                     0.809567   0.809567     0.809567
hits_at_3                     0.915936   0.915936     0.915936
hits_at_5                     0.947876   0.947876     0.947876
hits_at_10                    0.979189   0.979189     0.979189
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.895088   0.895088     0.895088
hits_at_1                     0.852515   0.852515     0.852515
hits_at_3                     0.924102   0.924102     0.924102
hits_at_5                     0.946575   0.946575     0.946575
hits_at_10                    0.971409   0.971409     0.971409
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.882745   0.882745     0.882745
hits_at_1                     0.831041   0.831041     0.831041
hits_at_3                     0.920019  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/10.3k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.148327   0.148327     0.148327
hits_at_1                     0.094103   0.094103     0.094103
hits_at_3                     0.159692   0.159692     0.159692
hits_at_5                     0.196185   0.196185     0.196185
hits_at_10                    0.254379   0.254379     0.254379
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.229602   0.229603     0.229602
hits_at_1                     0.147236   0.147236     0.147236
hits_at_3                     0.264694   0.264694     0.264694
hits_at_5                     0.319677   0.319677     0.319677
hits_at_10                    0.386726   0.386726     0.386726
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.188965   0.188965     0.188965
hits_at_1                     0.120670   0.120670     0.120670
hits_at_3                     0.212193  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/13.1k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.190184   0.190184     0.190184
hits_at_1                     0.135954   0.135954     0.135954
hits_at_3                     0.203395   0.203395     0.203395
hits_at_5                     0.242468   0.242468     0.242468
hits_at_10                    0.296834   0.296834     0.296834
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.273876   0.273876     0.273876
hits_at_1                     0.202783   0.202783     0.202783
hits_at_3                     0.304022   0.304022     0.304022
hits_at_5                     0.347989   0.347989     0.347989
hits_at_10                    0.406714   0.406714     0.406714
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.232030   0.232030     0.232030
hits_at_1                     0.169368   0.169368     0.169368
hits_at_3                     0.253709  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   

                                                value  
0                                              cuda:0  
1                                            526464

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)